## Loading the model from save state

Reconstructing the model

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

path = os.getcwd()
models_path = path + "\\Models"

reconstructed_model = keras.models.load_model(models_path)

Testing the reconstructed model on new data

In [20]:
import wapi # wattsights egen pakke / den må installers via `pip install wapi-python`
import pandas as pd
from datetime import datetime 
from datetime import timedelta

# Create a session
# Dette forutsetter at dere har lagret volue credentials som miljøvariabler på PC'en deres og med navnet `WS_CLIENT_ID` og `WS_CLIENT_SECRET`
user = os.getenv("WS_CLIENT_ID")
password = os.getenv("WS_CLIENT_SECRET")
session = wapi.Session(client_id=user, client_secret=password)

start = '2020-06-16'
end = '2020-09-16'
periods=pd.date_range(start,end,freq='D')

curves = ['con no1 mwh/h cet min15 a',
'tt no1 con ec00 °c cet min15 f',
'pro no1 wnd ec00 mwh/h cet min15 f',
'pro no1 spv ec00 mwh/h cet min15 f',
]



df_watt = pd.DataFrame()
for curve in curves:
    print(f"Working with {curve}")
    curve = session.get_curve(name=curve)
    curve_type = curve.curve_type
    res = []
    counter = 1
    if curve_type == 'TIME_SERIES':
        ts = curve.get_data(data_from=start, data_to=end, function='SAVERAGE', frequency='H')
        watt_temp = ts.to_pandas()
        df_watt = pd.merge(df_watt, watt_temp, left_index=True, right_index=True, how='outer')
        continue
    
    for j in range(len(periods)):
        if j == len(periods)-1:
            continue
        if curve_type == 'TAGGED_INSTANCES':
            ts_list = curve.search_instances(issue_date_from=periods[j],
                                 issue_date_to=periods[j+1],
                                 with_data=True, function='SAVERAGE', frequency='H', tags=['Avg'])
        else: #curve_type == 'INSTANCES'
            ts_list = curve.search_instances(issue_date_from=periods[j],
                                 issue_date_to=periods[j+1],
                                 with_data=True, function='SAVERAGE', frequency='H')
        # check if ts_list is empty and if yes, jump over that timestamp
        if not ts_list:
            continue
        pd_s = ts_list[0].to_pandas()
        if pd_s.empty:
            continue
        pd_s = pd_s.sort_index(ascending=True)
        pd_s = pd_s[:24]
        res.append(pd_s)
        print(f'Done {counter} out of {len(periods)}')
        counter += 1

    for k in range(len(res)):
        if k == 0:
            watt = pd.Series(res[k])
        else:
            watt = pd.concat([watt, res[k]])
    watt_temp = watt.rename(curve)
    df_watt = pd.merge(df_watt, watt_temp, left_index=True, right_index=True, how='outer')

df_watt


Working with con no1 mwh/h cet min15 a
Working with tt no1 con ec00 °c cet min15 f
Done 1 out of 93
Done 2 out of 93
Done 3 out of 93
Done 4 out of 93
Done 5 out of 93
Done 6 out of 93
Done 7 out of 93
Done 8 out of 93
Done 9 out of 93
Done 10 out of 93
Done 11 out of 93
Done 12 out of 93
Done 13 out of 93
Done 14 out of 93
Done 15 out of 93
Done 16 out of 93
Done 17 out of 93
Done 18 out of 93
Done 19 out of 93
Done 20 out of 93
Done 21 out of 93
Done 22 out of 93
Done 23 out of 93
Done 24 out of 93
Done 25 out of 93
Done 26 out of 93
Done 27 out of 93
Done 28 out of 93
Done 29 out of 93
Done 30 out of 93
Done 31 out of 93
Done 32 out of 93
Done 33 out of 93
Done 34 out of 93
Done 35 out of 93
Done 36 out of 93
Done 37 out of 93
Done 38 out of 93
Done 39 out of 93
Done 40 out of 93
Done 41 out of 93
Done 42 out of 93
Done 43 out of 93
Done 44 out of 93
Done 45 out of 93
Done 46 out of 93
Done 47 out of 93
Done 48 out of 93
Done 49 out of 93
Done 50 out of 93
Done 51 out of 93
Done 52 

,con no1 mwh/h cet min15 a,INSTANCES(tt no1 con ec00 °c cet min15 f),INSTANCES(pro no1 wnd ec00 mwh/h cet min15 f),INSTANCES(pro no1 spv ec00 mwh/h cet min15 f)
2020-06-15 23:00:00+02:00,NaN,19.9,33.172425,0.000000
2020-06-16 00:00:00+02:00,2549.0,19.2,27.898860,0.000000
2020-06-16 01:00:00+02:00,2356.0,18.7,24.666675,0.000000
2020-06-16 02:00:00+02:00,2227.0,18.1,21.774720,0.000000
2020-06-16 03:00:00+02:00,2159.0,17.2,18.712650,0.000000
...,...,...,...,...
2020-09-15 19:00:00+02:00,3436.0,14.6,6.294255,0.283921
2020-09-15 20:00:00+02:00,3433.0,13.6,8.165520,0.000000
2020-09-15 21:00:00+02:00,3265.0,13.0,11.397705,0.000000
2020-09-15 22:00:00+02:00,3051.0,12.8,14.629890,0.000000


In [21]:
df_watt.columns = ["cons_actual", "temp_forecast", "wind_forecast", "solar_forecast"]
df_watt.dropna(inplace=True)
df_watt

,cons_actual,temp_forecast,wind_forecast,solar_forecast
2020-06-16 00:00:00+02:00,2549.0,19.2,27.898860,0.000000
2020-06-16 01:00:00+02:00,2356.0,18.7,24.666675,0.000000
2020-06-16 02:00:00+02:00,2227.0,18.1,21.774720,0.000000
2020-06-16 03:00:00+02:00,2159.0,17.2,18.712650,0.000000
2020-06-16 04:00:00+02:00,2158.0,16.9,13.609200,0.000054
...,...,...,...,...
2020-09-15 18:00:00+02:00,3423.0,15.9,7.144830,3.869835
2020-09-15 19:00:00+02:00,3436.0,14.6,6.294255,0.283921
2020-09-15 20:00:00+02:00,3433.0,13.6,8.165520,0.000000
2020-09-15 21:00:00+02:00,3265.0,13.0,11.397705,0.000000


Preprocessing

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

In [23]:
df_watt['Seconds'] = df_watt.index.map(pd.Timestamp.timestamp)
day = 60*60*24
year = 365.2425*day
week = day*7

df_watt['Day sin'] = np.sin(df_watt['Seconds'] * (2* np.pi / day))
df_watt['Day cos'] = np.cos(df_watt['Seconds'] * (2 * np.pi / day))
df_watt['Week sin'] = np.sin(df_watt['Seconds'] * (2 * np.pi / week))
df_watt['Week cos'] = np.cos(df_watt['Seconds'] * (2 * np.pi / week))
df_watt['Year sin'] = np.sin(df_watt['Seconds'] * (2 * np.pi / year))
df_watt['Year cos'] = np.cos(df_watt['Seconds'] * (2 * np.pi / year))

df_watt=df_watt.drop(['Seconds'], axis=1)

df_watt

,cons_actual,temp_forecast,wind_forecast,solar_forecast,Day sin,Day cos,Week sin,Week cos,Year sin,Year cos
2020-06-16 00:00:00+02:00,2549.0,19.2,27.898860,0.000000,-5.000000e-01,8.660254e-01,-0.955573,-0.294755,0.268960,-0.963151
2020-06-16 01:00:00+02:00,2356.0,18.7,24.666675,0.000000,-2.588190e-01,9.659258e-01,-0.965926,-0.258819,0.268269,-0.963344
2020-06-16 02:00:00+02:00,2227.0,18.1,21.774720,0.000000,-2.858239e-12,1.000000e+00,-0.974928,-0.222521,0.267579,-0.963536
2020-06-16 03:00:00+02:00,2159.0,17.2,18.712650,0.000000,2.588190e-01,9.659258e-01,-0.982566,-0.185912,0.266888,-0.963728
2020-06-16 04:00:00+02:00,2158.0,16.9,13.609200,0.000054,5.000000e-01,8.660254e-01,-0.988831,-0.149042,0.266197,-0.963919
...,...,...,...,...,...,...,...,...,...,...
2020-09-15 18:00:00+02:00,3423.0,15.9,7.144830,3.869835,-8.660254e-01,-5.000000e-01,-0.930874,0.365341,-0.965157,-0.261672
2020-09-15 19:00:00+02:00,3436.0,14.6,6.294255,0.283921,-9.659258e-01,-2.588190e-01,-0.916562,0.399892,-0.965344,-0.260980
2020-09-15 20:00:00+02:00,3433.0,13.6,8.165520,0.000000,-1.000000e+00,-1.228297e-11,-0.900969,0.433884,-0.965531,-0.260288
2020-09-15 21:00:00+02:00,3265.0,13.0,11.397705,0.000000,-9.659258e-01,2.588190e-01,-0.884115,0.467269,-0.965717,-0.259596


In [24]:
cols = list(df_watt)[0:10]
#New dataframe with only training data - 4 columns
df_for_training = df_watt[cols].astype(float)
df_for_training

,cons_actual,temp_forecast,wind_forecast,solar_forecast,Day sin,Day cos,Week sin,Week cos,Year sin,Year cos
2020-06-16 00:00:00+02:00,2549.0,19.2,27.898860,0.000000,-5.000000e-01,8.660254e-01,-0.955573,-0.294755,0.268960,-0.963151
2020-06-16 01:00:00+02:00,2356.0,18.7,24.666675,0.000000,-2.588190e-01,9.659258e-01,-0.965926,-0.258819,0.268269,-0.963344
2020-06-16 02:00:00+02:00,2227.0,18.1,21.774720,0.000000,-2.858239e-12,1.000000e+00,-0.974928,-0.222521,0.267579,-0.963536
2020-06-16 03:00:00+02:00,2159.0,17.2,18.712650,0.000000,2.588190e-01,9.659258e-01,-0.982566,-0.185912,0.266888,-0.963728
2020-06-16 04:00:00+02:00,2158.0,16.9,13.609200,0.000054,5.000000e-01,8.660254e-01,-0.988831,-0.149042,0.266197,-0.963919
...,...,...,...,...,...,...,...,...,...,...
2020-09-15 18:00:00+02:00,3423.0,15.9,7.144830,3.869835,-8.660254e-01,-5.000000e-01,-0.930874,0.365341,-0.965157,-0.261672
2020-09-15 19:00:00+02:00,3436.0,14.6,6.294255,0.283921,-9.659258e-01,-2.588190e-01,-0.916562,0.399892,-0.965344,-0.260980
2020-09-15 20:00:00+02:00,3433.0,13.6,8.165520,0.000000,-1.000000e+00,-1.228297e-11,-0.900969,0.433884,-0.965531,-0.260288
2020-09-15 21:00:00+02:00,3265.0,13.0,11.397705,0.000000,-9.659258e-01,2.588190e-01,-0.884115,0.467269,-0.965717,-0.259596


In [25]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

df_for_training_scaled

array([[3.66666667e-01, 5.98173516e-01, 2.07858048e-01, ...,
        3.52622413e-01, 1.00000000e+00, 4.97217180e-02],
       [2.70646766e-01, 5.75342466e-01, 1.83776933e-01, ...,
        3.70590477e-01, 9.99440878e-01, 4.94619162e-02],
       [2.06467662e-01, 5.47945205e-01, 1.62230672e-01, ...,
        3.88739533e-01, 9.98881644e-01, 4.92027822e-02],
       ...,
       [8.06467662e-01, 3.42465753e-01, 6.08365019e-02, ...,
        7.16941870e-01, 3.01367755e-04, 9.98131932e-01],
       [7.22885572e-01, 3.15068493e-01, 8.49176172e-02, ...,
        7.33634314e-01, 1.50482980e-04, 9.99065876e-01],
       [6.16417910e-01, 3.05936073e-01, 1.08998733e-01, ...,
        7.50000000e-01, 0.00000000e+00, 1.00000000e+00]])

In [26]:
#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of timesteps we want to look into the future based on the past timesteps.
n_past = 24  # Number of past timesteps we want to use to predict the future.

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (2183, 24, 10).
trainY shape == (2183, 1).


In [31]:
predict_dates = pd.date_range(start, end, freq='H')
n_days_for_prediction = (pd.to_datetime(end) - pd.to_datetime(start)).days
n_days_for_prediction

92

In [39]:
#Make prediction
# predict_dates = predict_dates.reset_index(drop=True)
prediction = reconstructed_model.predict(predict_dates) #shape = (n, 1) where n is the n_days_for_prediction

TypeError: Cannot convert the argument `type_value`: dtype('<M8[ns]') to a TensorFlow DType.